In [ ]:
import pandas as pd
import numpy as np

# Nombre del archivo que está en la misma ruta
nombre_archivo = "Datos_Banos.xlsx"

# Cargar el archivo .xlsx en un DataFrame
try:
    df = pd.read_excel(nombre_archivo)
    
    # Mostrar las primeras 5 filas del DataFrame
    print(f"Archivo '{nombre_archivo}' cargado exitosamente.")
    print("---")
    print("Primeras 5 filas del DataFrame 'df':")
    print(df.head(5))

except FileNotFoundError:
    print(f"ERROR: El archivo '{nombre_archivo}' no se encontró en el directorio actual.")
    print("Asegúrate de que el archivo exista y esté en la misma ubicación que este cuaderno.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo: {e}")

Archivo 'Datos_Banos.xlsx' cargado exitosamente.
---
Primeras 5 filas del DataFrame 'df':
   Correlativo  Cod_bano Tipo_bano       Fecha            Proceso  \
0           77  77-B1-B6        B6  2025-04-22  ARMADO TABIQUES 6   
1           77  77-B1-B6        B6  2025-04-22  ARMADO TABIQUES 5   
2           77  77-B1-B6        B6  2025-04-22  ARMADO TABIQUES 4   
3           77  77-B1-B6        B6  2025-04-22  ARMADO TABIQUES 3   
4           77  77-B1-B6        B6  2025-04-22  ARMADO TABIQUES 2   

   T_Espera_min  T_Real_min  T_Real_Acumulado_min  TT  Cumple_TT  \
0           0.0       10.13                 10.13  20       True   
1           0.0        8.80                 18.93  20       True   
2           0.0       24.48                 43.42  20      False   
3           0.0       26.82                 70.23  20      False   
4           0.0       28.05                 98.28  20      False   

   Diferencia_TT  Porcentaje_TT Operario_1 Operario_2 Operario_3  \
0          -9.87  

In [ ]:
# 1. Limpieza de Operarios (Usaremos 'N/A' si el campo Operario_n fue NaN, para insertarlo como operario desconocido)
df_temp = df[['Operario_1', 'Operario_2', 'Operario_3']].copy()
df_temp = df_temp.fillna('N/A')


df[['id_b_str', 'edificio', 'variante']] = df['Cod_bano'].str.split('-', expand=True)


try:
    df['id_b'] = df['Correlativo'] # El ID del baño en la tabla `bano` es `id_b`
    df['edificio'] = df['Cod_bano'].str.extract(r'-([A-Z])(\d+)-').iloc[:, 0]
    df['piso'] = df['Cod_bano'].str.extract(r'-([A-Z])(\d+)-').iloc[:, 1].astype(int)
    df['variante'] = df['Tipo_bano'] # 'B6'
except Exception as e:
    print(f"ERROR al parsear Cod_bano: {e}. Asumiendo que el formato Cod_bano es <id_b>-<edificio><piso>-<variante>")
    # Si el formato es '77-B1-B6': id_b=77, edificio=B, piso=1, variante=B6.
    df['id_b'] = df['Cod_bano'].str.split('-').str[0].astype(int)
    df['edificio'] = df['Cod_bano'].str.split('-').str[1].str[0]
    df['piso'] = df['Cod_bano'].str.split('-').str[1].str[1].astype(int)
    df['variante'] = df['Cod_bano'].str.split('-').str[2]
    
# 3. Conversión de Cumple_TT a TINYINT(1) (0 o 1)
df['Cumple_TT_INT'] = df['Cumple_TT'].astype(int)

print("Preprocesamiento completado. Componentes de Baño y Cumple_TT extraídos/convertidos.")

✅ Preprocesamiento completado. Componentes de Baño y Cumple_TT extraídos/convertidos.


In [ ]:
# Definición de la función de formato SQL
def format_sql_value(val):
    """
    Convierte un valor de Pandas (int, float, string, NaN, datetime, bool) 
    a una cadena de texto adecuada para una sentencia SQL INSERT.
    """
    if pd.isna(val):
        return 'NULL'
    if isinstance(val, str):
        # Escapar comillas y encerrar en comillas simples
        return f"'{val.replace("'", "''")}'"
    if pd.api.types.is_datetime64_any_dtype(val):
        # Formato estándar de fecha para SQL ('YYYY-MM-DD')
        return f"'{val.strftime('%Y-%m-%d')}'"
    if isinstance(val, bool):
        return '1' if val else '0'
    return str(val)

# --- CONFIGURACIÓN DEL SCRIPT SQL ---
NOMBRE_ARCHIVO_SQL = 'insert_data_final.sql'

if 'df' in locals():
    
    with open(NOMBRE_ARCHIVO_SQL, 'w', encoding='utf-8') as f:
        f.write("-- Script de Inserción de Datos Generado desde Jupyter\n")
        f.write("-- Omite las tablas 'operario' y 'proceso' que ya están pobladas.\n")
        f.write("USE axis_bd;\n\n")

        # ------------------------------------------------------------
        # 1. INSERCIÓN: bano
        # ------------------------------------------------------------
        df_bano = df[['id_b', 'edificio', 'piso', 'variante']].drop_duplicates()
        f.write("-- ============================================================\n")
        f.write("-- 1. INSERCIÓN: bano\n")
        f.write("-- ============================================================\n")
        for index, row in df_bano.iterrows():
            insert_bano = f"INSERT INTO bano (id_b, variante, edificio, piso) VALUES ({row['id_b']}, {format_sql_value(row['variante'])}, {format_sql_value(row['edificio'])}, {row['piso']}) ON DUPLICATE KEY UPDATE id_b=id_b;\n"
            f.write(insert_bano)

        # ------------------------------------------------------------
        # 2. INSERCIÓN: ejecucion_proceso
        # ------------------------------------------------------------
        f.write("\n-- ============================================================\n")
        f.write("-- 2. INSERCIÓN: ejecucion_proceso\n")
        f.write("-- Obtiene id_proc de la tabla 'proceso' ya existente.\n")
        f.write("-- ============================================================\n")
        
        for index, row in df.iterrows():
            # Subconsulta para id_proc
            subquery_proc = f"(SELECT id_proc FROM proceso WHERE nom_proc = {format_sql_value(row['Proceso'])} AND tt_proc = {row['TT']})"
            
            insert_ejec = f"""
INSERT INTO ejecucion_proceso (id_b, id_proc, fecha, edificio, piso, variante, tt_proc, t_real_min, t_espera_min, t_real_acum_min) 
VALUES (
    {row['id_b']}, 
    {subquery_proc}, 
    {format_sql_value(row['Fecha'])}, 
    {format_sql_value(row['edificio'])}, 
    {row['piso']}, 
    {format_sql_value(row['variante'])}, 
    {row['TT']}, 
    {row['T_Real_min']}, 
    {row['T_Espera_min']}, 
    {row['T_Real_Acumulado_min']}
);
"""
            f.write(insert_ejec.strip() + '\n')

        # ------------------------------------------------------------
        # 3. INSERCIÓN: ejecucion_operario
        # ------------------------------------------------------------
        f.write("\n-- ============================================================\n")
        f.write("-- 3. INSERCIÓN: ejecucion_operario (N:M)\n")
        f.write("-- La inserción es OMITIDA si la sigla del operario está vacía ('') o es NaN.\n")
        f.write("-- ============================================================\n")

        for index, row in df.iterrows():
            # Subconsulta para id_ejec (Identifica el registro recién insertado)
            subquery_ejec = f"""
(SELECT id_ejec FROM ejecucion_proceso 
 WHERE id_b = {row['id_b']} 
   AND fecha = {format_sql_value(row['Fecha'])} 
   AND id_proc = (SELECT id_proc FROM proceso WHERE nom_proc = {format_sql_value(row['Proceso'])} AND tt_proc = {row['TT']})
   AND t_real_min = {row['T_Real_min']} 
 ORDER BY id_ejec DESC LIMIT 1)
"""
            
            for i in range(1, 4):
                sigla_op = row[f'Operario_{i}']
                rol_op = f'Operario_{i}'
                
                if pd.notna(sigla_op) and sigla_op != '':

                    subquery_op = f"(SELECT id_op FROM operario WHERE sigla_op = {format_sql_value(sigla_op)})"
                    
                    insert_ejec_op = f"""
INSERT INTO ejecucion_operario (id_ejec, id_op, rol) 
VALUES (
    {subquery_ejec}, 
    {subquery_op}, 
    {format_sql_value(rol_op)}
);
"""
                    f.write(insert_ejec_op.strip() + '\n')
                    
    print(f"\nScript de inserción generado con éxito en: '{NOMBRE_ARCHIVO_SQL}'")
    print("La lógica de omisión de operarios nulos/vacíos ha sido aplicada.")
else:
    print("El DataFrame 'df' no fue cargado en la celda anterior. Verifique si el archivo existe.")


✅ Script de inserción generado con éxito en: 'insert_data_final.sql'
La lógica de omisión de operarios nulos/vacíos ha sido aplicada.
